In [ ]:
import os, sys
import matplotlib.pyplot as plt
import numpy as np

# Biblioteca y módulos de flopy
import flopy
from flopy.plot.styles import styles

# Extras para mf6 y flopy (módulo creado en este proyecto)
import xmf6

from modflowapi import ModflowApi

In [ ]:
init = {
    'sim_name' : "flow",
    'exe_name' : "C:\\Users\\luiggi\\Documents\\GitSites\\xmf6\\mf6\\windows\\mf6",
#    'exe_name' : "../../mf6/macosarm/mf6",
    'sim_ws' : os.path.join(".", "sandbox1")
}

time = {
    'units': "seconds",
    'nper' : 1,
    'perioddata': [(120.0, 1, 1.0)]
}

ims = {}

gwf = { 
    'modelname': init["sim_name"],
    'model_nam_file': f"{init["sim_name"]}.nam",
    'save_flows': True
}

o_sim, o_tdis, o_ims = xmf6.gwf.init_sim(silent = True, init = init, tdis = time, ims = ims)

In [ ]:
dis = {
    'length_units' : "centimeters",
    'nlay': 1, 
    'nrow': 1, 
    'ncol': 120,
    'delr': 0.1, 
    'delc': 0.1,
    'top' : 1.0, 
    'botm': 0.0 
}

ic = {
    'strt': 1.0
}

npf = {
    'save_specific_discharge': True,
    'save_saturation' : True,
    'icelltype' : 0,
    'k' : 0.01,
}

chd = {
    'stress_period_data': [[(0, 0, dis['ncol'] - 1), 1.0]],     
}

## Physical parameters
specific_discharge = 0.1  # Specific discharge ($cm s^{-1}$)
source_concentration = 1.0  # Source concentration (unitless)
q   = specific_discharge * dis['delc'] * dis['delr'] * dis['top']
aux = source_concentration

well = {
    'stress_period_data': [[(0, 0, 0), q, aux,]],
    'pname': "WEL-1",
    'save_flows': True,
    'auxiliary' : ["CONCENTRATION"],

}

oc = {
    'budget_filerecord': f"{init['sim_name']}.bud",
    'head_filerecord': f"{init['sim_name']}.hds",
    'saverecord' : [("HEAD", "ALL"), ("BUDGET", "ALL")],

}

o_gwf, _ = xmf6.gwf.set_packages(o_sim, silent = True,
                                 gwf = gwf, dis = dis, ic = ic, chd = chd, npf = npf, oc = oc, well = well)

o_sim.write_simulation(silent = True)

In [ ]:
if sys.platform == "win32":
    mf6_dll = "libmf6.dll"
elif sys.platform == "darwin":
    mf6_dll = "libmf6.dylib"
else:
    mf6_dll = "libmf6.so"

mf6_dll = os.path.abspath(os.path.join("..", "bin", mf6_dll))

mf6_config_file = os.path.join(init['sim_ws'], 'mfsim.nam')
mf6 = ModflowApi(mf6_dll, working_directory=init['sim_ws'])

# initialize the model
mf6.initialize(mf6_config_file)

# time loop
start_time = mf6.get_start_time()
current_time = mf6.get_current_time()
time_step = mf6.get_time_step()
end_time = mf6.get_end_time()

# maximum outer iterations
max_iter = mf6.get_value(mf6.get_var_address("MXITER", "SLN_1"))

In [ ]:
print(sys.platform, ": ", mf6_dll)
print("Config file:", mf6_config_file)
print("Start time:", start_time)
print("Current time:", current_time)
print("Time step:", time_step)
print("End time:", end_time)
print("Max. iter:", max_iter)

In [ ]:
# model time loop
idx = 0
current_time = 0
while current_time < end_time:
    current_time += 120
    print(current_time, end = " ")

    mf6.prepare_solve()
    kiter = 0
    while kiter < max_iter:
        has_converged = mf6.solve(1)
        print("Iter :", kiter)
        kiter += 1
        
        if has_converged:
            print(f" Convergencia obtenida : {has_converged}")
            break

    # finalize time step
    mf6.finalize_solve()

    # finalize time step and update time
#    mf6.finalize_time_step()
#    current_time = mf6.get_current_time()

    if not has_converged:
        print("model did not converge")
        break
# cleanup
try:
    mf6.finalize()
    success = True
except:
    raise RuntimeError
    

In [ ]:
mf6.get_var_address("XC", 'FLOW', "DIS")

In [ ]:
print("MF6 Input var names :", mf6.get_input_var_names()) # mf6.get_output_var_names()

In [ ]:
mf6.get_value(mf6.get_var_address("BOTM", '__INPUT__/FLOW', "DIS"))

In [ ]:
for label in mf6.get_input_var_names():
    if 'SLN' in label:
        print(label)

In [ ]:
mf6.get_value(mf6.get_var_address("MXITER", 'SLN_1'))

In [ ]:
for label in mf6.get_input_var_names():
    if 'FLOW/DIS' in label:
        print(label)

In [ ]:
for label in mf6.get_input_var_names():
    if 'FLOW/DIS' in label:
        var_list = label.split('/')
        if '__INPUT__' in label and 'DISENUM' not in label:
            comp_name = '/'.join((var_list[0], var_list[1]))
            scomp_name = var_list[2]
        else:
            comp_name = var_list[0]
            scomp_name = var_list[1]

        var_value = mf6.get_value(mf6.get_var_address(var_list[-1], comp_name, scomp_name))
        print(label, var_value)

In [ ]:
for label in mf6.get_input_var_names():
    if 'FLOW/CHD' in label:
        print(label)

In [ ]:
fail = ["FLOW/CHD_0/BOUNDNAME", 'FLOW/CHD_0/AUXNAME']
for label in mf6.get_input_var_names():
    if 'FLOW/CHD' in label and label not in fail :
        var_list = label.split('/')
        if '__INPUT__' in label:
            comp_name = '/'.join((var_list[0], var_list[1]))
            scomp_name = var_list[2]
        else:
            comp_name = var_list[0]
            scomp_name = var_list[1]

        var_value = mf6.get_value(mf6.get_var_address(var_list[-1], comp_name, scomp_name))
        print(label, var_value)

In [ ]:


dis_address = mf6.get_var_address("NDIM", init['sim_name'].upper(), "DIS")
dim = mf6.get_value(dis_address)
print("NDIM:", dim)

print(mf6.get_value(mf6.get_var_address("YC", init['sim_name'].upper(), "DIS")))

print(mf6.get_value(mf6.get_var_address("X", init['sim_name'].upper())))


In [ ]:
o_sim.run_simulation()

In [ ]:
# --- Recuperamos los resultados de la simulación ---
head = xmf6.gwf.get_head(o_gwf)
qx, qy, qz, n_q = xmf6.gwf.get_specific_discharge(o_gwf)

# Verificamos el tipo y dimensiones de los arreglos donde
# están almacenados la carga hidráulica, el BUDGET, y la velocidad.
print('Head : ', type(head), head.shape)
print('qx : ', type(qx), qx.shape)
print('qy : ', type(qy), qy.shape)
print('qz : ', type(qz), qz.shape)

print(head.shape, '\n', head)

In [ ]:
# Obtenemos el objeto con la información de la discretización espacial
grid = o_gwf.modelgrid

# Coordenadas del centro de celdas para graficación
x, _, _ = grid.xyzcellcenters
print(x)
with styles.USGSPlot():
    plt.rcParams['font.family'] = 'DeJavu Sans'
    plt.figure(figsize=(10,3))
    plt.plot(x[0], head[0, 0], marker=".", ls ="-", mec="blue", mfc="none", markersize="1", label = 'Head')
    plt.xlim(0, 12)
    plt.xticks(ticks=np.linspace(0, grid.extent[1],13))
    plt.xlabel("Distance (cm)")
    plt.ylabel("Head")
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
plt.figure(figsize=(10,0.15))
ax = plt.gca()
pmv0 = flopy.plot.PlotMapView(o_gwf, ax=ax)
pmv0.plot_grid(colors='dimgray', lw=0.5)
plt.show()